In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time
import requests

# Setup ChromeDriver
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

# URL of the page to scrape
url = 'https://www.vtb.ru/personal/vklady-i-scheta/obezlichennyj-metallicheskij-schet/'

sell_price_ru_rub = 0;
buy_price_ru_rub = 0;

try:
    # Navigate to the page
    driver.get(url)
    
    # Wait for the dynamic content to load
    time.sleep(3)  # Adjust the sleep time if necessary
    
    # Find all price data elements using their class (adjust if necessary)
    price_elements = driver.find_elements(By.CSS_SELECTOR, 'p.typographystyles__Box-foundation-kit__sc-14qzghz-0.jEFSaq.numbersstyles__TypographyTitle-foundation-kit__sc-1xhbrzd-4.haHdlc')
    
    # Assuming the first element is the sell price and the second is the buy price
    if len(price_elements) >= 2:
        sell_price_ru_rub = price_elements[0].text
        buy_price_ru_rub = price_elements[1].text
        print(f'Sell price: {sell_price_ru_rub}')
        print(f'Buy price: {buy_price_ru_rub}')
    else:
        print('Price data not found.')
finally:
    # Close the browser
    driver.quit()


In [ ]:
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

# URL of the page to scrape
url = 'https://bullionexchanges.com/sell-to-us/bars'

buy_price_usa = 0;
sell_price_usa = 0;

try:
    # Navigate to the page
    driver.get(url)
    
    # Wait for the dynamic content to load
    time.sleep(3)  # Adjust the sleep time if necessary
    
    # Find all price data elements using their class
    price_elements = driver.find_elements(By.CLASS_NAME, 'price-alZZ')
    
    # Assuming the first element is the buy price and the second is the sell price
    if len(price_elements) >= 2:
        buy_price_usa = price_elements[0].text
        sell_price_usa = price_elements[1].text
        print(f'Buy price: {buy_price_usa}')
        print(f'Sell price: {sell_price_usa}')
    else:
        print('Price data not found.')
finally:
    # Close the browser
    driver.quit()


In [ ]:
# Conversion factor from ounces to grams
oz_to_grams = 28.3495

# Convert the string prices to floats
buy_price_usa = float(buy_price_usa)
sell_price_usa = float(sell_price_usa)

def convert_price_oz_to_gram(price_per_oz):
    """Converts price from per ounce to per gram."""
    return price_per_oz / oz_to_grams

# Convert buy and sell prices to per gram
buy_price_usa_gram_usd = convert_price_oz_to_gram(buy_price_usa)
sell_price_usa_gram_usd = convert_price_oz_to_gram(sell_price_usa)

print(f'Buy price per gram: ${buy_price_usa_gram_usd:.2f}')
print(f'Sell price per gram: ${sell_price_usa_gram_usd:.2f}')

In [ ]:
def get_usd_rub_exchange_rate():
    # URL for the ExchangeRate-API to get the latest exchange rates
    url = "https://api.exchangerate-api.com/v4/latest/USD"

    try:
        # Make a GET request to the API
        response = requests.get(url)
        response.raise_for_status()  # Raises an HTTPError if the response status code is 4XX/5XX

        # Parse the response JSON
        data = response.json()

        # Get the USD to RUB exchange rate
        usd_to_rub = data['rates']['RUB']
        return usd_to_rub
    except requests.RequestException as e:
        print(f"Error fetching exchange rate: {e}")
        return None

# Example usage
exchange_rate = get_usd_rub_exchange_rate()
if exchange_rate is not None:
    print(f"USD/RUB exchange rate: {exchange_rate}")
else:
    print("Failed to retrieve the exchange rate.")

buy_price_ru_rub_normalized = float(buy_price_ru_rub.replace(" ", "").replace(",", "."))

# Convert the cost of buying in Russia to USD
cost_in_usd = buy_price_ru_rub_normalized / exchange_rate

# Calculate P&L per gram
profit_per_gram_usd = buy_price_usa_gram_usd - cost_in_usd

print("Price per gram AU in Russia:", "${:.2f}".format(buy_price_ru_rub_normalized/exchange_rate))
print("Price per gram AU in USA:", "${:.2f}".format(sell_price_usa_gram_usd))
print("Profit per gram in USD:", "${:.2f}".format(profit_per_gram_usd))